In [3]:
# # Nếu chưa có thư viện thì run cell này
# !pip install mediapipe
# !pip install pandas
# !pip install tensorflow
# !pip install numpy

# 1. Import and Install Dependencies

In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(
        image
        )                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [7]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*4)
    rh = np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*4)
    return np.concatenate([pose, lh, rh])

# 3. Setup Folders for Collection

In [9]:
actions = np.array(
    ['Bye', 'Fine', 'Good', 'Hello', 'I', 'You', 'Love', '_']
    )
WEIGHT_PATH = os.path.join('./weightsnewdata.h5')

sequence_length = 10


# 4. Test in Real Time

In [10]:
import tensorflow as tf

2023-04-22 16:50:24.209508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-22 16:50:24.465691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nhamcotdo/anaconda3/lib/python3.7/site-packages/cv2/../../lib64:
2023-04-22 16:50:24.465721: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-22 16:50:26.646788: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [11]:
actions = np.array(
    ['Bye', 'Fine', 'Good', 'Hello', 'I', 'You', 'Love', '_']
    )
WEIGHT_PATH = os.path.join('./weightsnewdata.h5')

sequence_length = 10


In [12]:
tf.__version__

'2.11.0'

In [13]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res[:6]):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    for num, prob in enumerate(res[7:]):
        cv2.rectangle(output_frame, (input_frame.shape[1]-120,60+num*40), (input_frame.shape[1]-120+int(prob*100), 90+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num+6], (input_frame.shape[1]-120, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [14]:
model = tf.keras.models.load_model(WEIGHT_PATH)

2023-04-22 16:50:38.184874: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nhamcotdo/anaconda3/lib/python3.7/site-packages/cv2/../../lib64:
2023-04-22 16:50:38.184927: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-22 16:50:38.184957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nhamcotdo): /proc/driver/nvidia/version does not exist
2023-04-22 16:50:38.185289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate c

In [19]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.6

# cap = cv2.VideoCapture('/home/nhamcotdo/Downloads/Phân-20230409T104504Z-001/')
cap = cv2.VideoCapture('/home/nhamcotdo/Downloads/Phân-20230409T104504Z-001/Phân/NhieuHanhDong/2023-04-09 15_02_28.avi')
fps = cap.get(cv2.CAP_PROP_FPS)
print("FPS:", fps)
# cap = cv2.VideoCapture(0)
# Set mediapipe model
try:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # if not ret:
            #     cv2.destroyAllWindows()
            #     break

            image, results = mediapipe_detection(frame, holistic)
            # Make detections
            # Draw landmarks
            # draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-sequence_length:]

            if len(sequence) == sequence_length:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                # print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))

                if np.unique(predictions[-10:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:

                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                print(sentence)
                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)

            cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
except:
    cap.release()
    cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()


FPS: 120.0


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/nhamcotdo/anaconda3/lib/python3.7/site-packages/cv2/qt/plugins"
QObject::moveToThread: Current thread (0x5594e23038e0) is not the object's thread (0x5594e233cfe0).
Cannot move to target thread (0x5594e23038e0)

QObject::moveToThread: Current thread (0x5594e23038e0) is not the object's thread (0x5594e233cfe0).
Cannot move to target thread (0x5594e23038e0)

QObject::moveToThread: Current thread (0x5594e23038e0) is not the object's thread (0x5594e233cfe0).
Cannot move to target thread (0x5594e23038e0)

QObject::moveToThread: Current thread (0x5594e23038e0) is not the object's thread (0x5594e233cfe0).
Cannot move to target thread (0x5594e23038e0)

QObject::moveToThread: Current thread (0x5594e23038e0) is not the object's thread (0x5594e233cfe0).
Cannot move to target thread (0x5594e23038e0)

QObject::moveToThread: Current thread (0x5594e23038e0) is not the object's thread (0x5594e233cfe0).
Cannot move to target thread

1/1 [==============================] - 1s 762ms/step
['_']
1/1 [==============================] - 0s 45ms/step
['_']
1/1 [==============================] - 0s 35ms/step
['_']
1/1 [==============================] - 0s 27ms/step
['_']
1/1 [==============================] - 0s 35ms/step
['_']
1/1 [==============================] - 0s 45ms/step
['_']
1/1 [==============================] - 0s 38ms/step
['_']
1/1 [==============================] - 0s 43ms/step
['_']
1/1 [==============================] - 0s 29ms/step
['_']
1/1 [==============================] - 0s 45ms/step
['_']
1/1 [==============================] - 0s 34ms/step
['_']
1/1 [==============================] - 0s 31ms/step
['_']
1/1 [==============================] - 0s 38ms/step
['_']
1/1 [==============================] - 0s 38ms/step
['_']
1/1 [==============================] - 0s 34ms/step
['_']
1/1 [==============================] - 0s 40ms/step
['_']
1/1 [==============================] - 0s 40ms/step
['_']
1/1 [========